<a href="https://colab.research.google.com/github/OswaldoLopezAlcaraz/TI_1_Practica4/blob/main/TI_1_Practica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Oswaldo López Alcaraz

**e-mail:** oswaldo.lopez5103@alumnos.udg.mx

# MODULES

In [2]:
import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

import pandas as pd
import numpy as np

import plotly.graph_objects as go

import math
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

# CLASSES

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# FUNCTIONS

In [10]:
###################### Function to create Brownian walks #######################
def bm_2d(n_steps=1000, speed=6, s_pos=[0,0]):
  """
  Summary:
    This function generates a Brownian Walk.
  Arguments:
    n_steps: number of steeps that comprises the entire walk.
    speed: number of units per steps.
    s_pos: coordinates (x and y ) where the walk starts.
  Returns:
    BM_2d_df: dataframe with the walk in form of coordinates.
  """
  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[i]+velocity.x, 'y_pos':BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df


################## Function to create Correlated Random Walks ##################
def crw_2d(n_steps=1000, speed=6, s_pos=[0,0], crw_exp=0.3):
  """
  Summary:
    This function generates a Correlated Random Walk.
  Arguments:
    n_steps: number of steeps that comprises the entire walk
    speed: number of units per steps
    s_pos: coordinates (x and y ) where the walk starts.
    crw_exp:  exponent used in the Cauchy distribution
  Returns:
    CRW_2d_df: dataframe with the walk in form of coordinates.
  """
  # Init of velocity vector
  velocity = Vec2d(speed,0)

  CRW_2d_df = pd.DataFrame( columns= ['x_pos','y_pos'])
  temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos': s_pos[1]},])
  CRW_2d_df = pd.concat([CRW_2d_df, temp_df], ignore_index=True)

  # Brownian walker in motion and assignation of directions
  for i in range(1, n_steps):

    # Determination of angle using cauchy distribution
    turn_angle = wrapcauchy.rvs(crw_exp)
    velocity = velocity.rotated(turn_angle)

    # Motion of Brownian walker
    temp_df = pd.DataFrame([{'x_pos':CRW_2d_df.x_pos[i-1]+velocity.x,
                           'y_pos':CRW_2d_df.y_pos[i-1]+velocity.y,}])
    CRW_2d_df = pd.concat([CRW_2d_df,temp_df], ignore_index=True)

  return CRW_2d_df


########## Function to obtain Euclidian distance between two points ############
def my_distance_function(origin, destination):
  """
  Arguments:
    origin: x & y coordinates that represent the origin of a trajectory
    destination: x & y coordinate representing the destination of a trajectory
  Returns:
    Euclidiean distance between two points in the form of int or float.
  """
  return math.sqrt(((destination.x_pos - origin.x_pos)**2) +
                    (( destination.y_pos - origin.y_pos)**2) )

################# Function to get Squared Mean Displacement ####################
def smd(motion_df):
  """
  Arguments:
    motion_df: dataframe with coordinates that represents a certain type of displacement.
  Returns:
    dataframe of length motion.df.shape[0] - 1 containing SMD in for each step
  """
  smd = pd.DataFrame(columns=['MSD'])

  for n in range(1,motion_df.shape[0]):

    displacement_vec = np.array([(my_distance_function(motion_df.iloc[i-n], motion_df.iloc[i]))**2
                              for i in range(n, motion_df.shape[0],1)])
    tmp_df = pd.DataFrame([{'MSD':np.mean(displacement_vec)}])
    smd = pd.concat([smd,tmp_df], ignore_index=True)

  return smd


##################### Function to create Brownian walks ########################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array([pos_b.x_pos - pos_a.x_pos , pos_b.y_pos - pos_a.y_pos ])
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array([pos_c.x_pos - pos_b.x_pos , pos_c.y_pos - pos_b.y_pos ])
    norm_bc = np.linalg.norm(vec_bc)

    dot_p =np.dot(vec_ab , vec_bc)

    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta

########################Function to automate Turning angles processes #########
def turning_angles_producer(motion_df):
  """
  Arguments:
    motion_df: dataframe with coordinates that represents a certain type of displacement.
  Returns:
    dataframe with turning angles for each step.
  """
  trn_angle_df = pd.DataFrame(columns = ['TA_CRW'])

  for i in range(motion_df.shape[0]-2):
    ta = turning_angle(motion_df.iloc[i], motion_df.iloc[i+1], motion_df.iloc[i+2])
    tmp_df = pd.DataFrame([{'TA_CRW': ta }])
    trn_angle_df = pd.concat([trn_angle_df, tmp_df], ignore_index=True)

  return trn_angle_df


 ############# Function to get the Step Length metrics of any walk #############
def step_length_determiner(motion_df):

    ## Init of variables
  step_lengths_df = pd.DataFrame(columns = ['count','Turn_angle','Step_length'])
  is_TA_zero = False
  steps_total = motion_df.shape[0] -3
  passed_ta = 0
  overall_steps = 0
  big_index = 0
  temp_steps_accum = 0
  count_index = 0
  # Try that will catch the IndexError when algorithm goes out of index limits.
  try:
    # First iteration through all the points.
    while big_index <= steps_total:
      # First angle check.
      t_a = turning_angle(motion_df.iloc[big_index],
                            motion_df.iloc[big_index +1],
                            motion_df.iloc[big_index + 2])
      is_TA_zero = t_a == 0.00
      # Verifying that a step needs to be added and "virtual counts" resets
      if not is_TA_zero:

        temp_steps_accum += 1
        tmp_df = pd.DataFrame([{'count': count_index ,
                              'Turn_angle':t_a,
                              'Step_length': temp_steps_accum}])
        step_lengths_df = pd.concat([step_lengths_df,tmp_df],ignore_index=True)
        temp_steps_accum = 0
        count_index += 1
        passed_ta = t_a
        big_index +=1
        overall_steps += 1

      else:  ## Following is the code used when there an zero angle is present.
        temp_steps_accum += 1
        overall_steps += 1
        while is_TA_zero:  # Second iteration corresponding to streight line walk

          t_a = turning_angle(motion_df.iloc[big_index + temp_steps_accum],
                              motion_df.iloc[big_index + temp_steps_accum + 1],
                              motion_df.iloc[big_index + temp_steps_accum + 2])
          is_TA_zero = t_a == 0.00

          ## block of code that verify the condition of second iterations continues
          ## steps are added virtualy in the second iteration.
          if is_TA_zero:
            temp_steps_accum += 1
            overall_steps += 1
          else:  ## If an angle bigger than zero comes, big index is adjust for the
                 ## first iteration
            big_index += temp_steps_accum
            passed_ta = t_a
  except IndexError:
    return step_lengths_df




##################### Function to create levy flights ##########################

def levy_2d(n_steps=1000,speed=1, s_x_pos=0, s_y_pos=0, levy_coeff=1.1, cauchy_coeff=0.3):
  # Determination of number of steps in a certain direction
  generated_steps = 0
  same_direction_steps = np.empty((1,1), dtype=int)
  vindex = 0
  correction_factor = 0
  while generated_steps <= n_steps:
    xstep_num = abs(levy_stable.rvs(1.1, 0, loc=1,size=1))
    step_num = np.around(xstep_num)
    if vindex == 0:
      same_direction_steps[0] = step_num
      vindex = 1
    else:
      same_direction_steps = np.append(same_direction_steps, step_num)
    generated_steps = same_direction_steps.sum()
  #Adjustment in last cell to limit the total steps to the desired number
  correction_factor = generated_steps - n_steps
  same_direction_steps[-1]= same_direction_steps[-1] - correction_factor

  # Initialization of Brownian Walker Matrix with pandas
  BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos'])
  tmp_df = pd.DataFrame([{'x_pos':s_x_pos, 'y_pos': s_y_pos}])
  BM_2d_df = pd.concat([BM_2d_df,tmp_df], ignore_index=True)

  # Brownian walker in motion and assignation of directions
  index_count=0
  for direction in same_direction_steps:
    velocity = Vec2d(speed,0)
    # Determination of angle using Cauchy distribution
    turn_angle = abs(wrapcauchy.rvs(cauchy_coeff))
    velocity = velocity.rotated(turn_angle)
    x_velocity = velocity.x
    y_velocity = velocity.y

    for step in range(0,int(direction)):
      # Motion of Brownian walker
      tmp_df = pd.DataFrame([{'x_pos':BM_2d_df.x_pos[index_count] + x_velocity,
                              'y_pos':BM_2d_df.y_pos[index_count]+y_velocity}])
      BM_2d_df = pd.concat([BM_2d_df,tmp_df], ignore_index=True)
      index_count += 1

  return BM_2d_df

# DASHBOARD

In [5]:

# Main Scaffold using box
column_a = pn.Column('Row_A', 'Row_B', 'Row_C')
row_a = pn.Row(column_a, 'Columna_B', 'Columna_C')



In [6]:
# Creation of RW selector
walks_radio_btn = pnw.RadioBoxGroup(name='Trajectories', value='BM' , options=['BM', 'CWR', 'LF'], inline=True, width=130)
rw_type_selector_container = pn.WidgetBox('### RW Type',walks_radio_btn)

In [7]:
# Creation of metrics
metrics_type = pnw.Select(name='Metrics Selection', value='PL',options=['PL','SMD'], width=130)
metrics_type_container = pn.WidgetBox('### Metrics Type',metrics_type)

In [9]:
# Creation of parameters

n_steps = pnw.IntInput(name='Number of steps', value=50, step=10, start=1, end=1000, width=130)
speed = pnw.IntInput(name='Speed', value=0, step=1, start=1, end=10, width=130)
s_x_pos = pnw.IntInput(name='Starting pos X', value=0, step=1, start=-100, end=100, width=130)
s_y_pos = pnw.IntInput(name='Starting pos y', value=0, step=1, start=-100, end=100, width=130)
cauchy_coeff = pnw.FloatSlider(name='Cauchy coefficient', value=0.1, step=0.01, start=0.01, end=0.99, width=130)
levy_coeff = pnw.FloatSlider(name='Levy coefficient', value=0.1, step=0.01, start=0.01, end=0.99, width=130, visible=True)
parameters_container = pn.WidgetBox("### Parameters", n_steps,speed,s_x_pos,s_y_pos,cauchy_coeff,levy_coeff)
column_a = pn.Column(rw_type_selector_container, metrics_type_container, parameters_container)
row_a = pn.Row(column_a, 'Columna_B', 'Columna_C')
row_a


Row
    [0] Column
        [0] WidgetBox
            [0] Markdown(str)
            [1] RadioBoxGroup(inline=True, name='Trajectories', options=['BM', 'CWR', 'LF'], value='BM', width=130)
        [1] WidgetBox
            [0] Markdown(str)
            [1] Select(name='Metrics Selection', options=['PL', 'SMD'], value='PL', width=130)
        [2] WidgetBox
            [0] Markdown(str)
            [1] IntInput(end=1000, name='Number of steps', start=1, step=10, value=50, width=130)
            [2] IntInput(end=10, name='Speed', start=1, width=130)
            [3] IntInput(end=100, name='Starting pos X', start=-100, width=130)
            [4] IntInput(end=100, name='Starting pos y', start=-100, width=130)
            [5] FloatSlider(end=0.99, name='Cauchy coefficient', start=0.01, step=0.01, value=0.1, width=130)
            [6] FloatSlider(end=0.99, name='Levy coefficient', start=0.01, step=0.01, value=0.1, width=130)
    [1] Markdown(str)
    [2] Markdown(str)